# Experiment 10
- **Model:** `yolov8n` *(Nano)*
- **Dataset:** 5m | 60º
- **Sizes:** large
- **Experiments:**
    1. large


## Init

In [1]:
import os
import shutil
import fnmatch

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

## Helper Functions

In [3]:
# Change for different file formats
reference = {
  "small": {
    "suffix": ".S",
    "file": "209"
  },
  "mid": {
    "suffix": ".M",
    "file": "503"
  },
  "large": {
    "suffix": ".L",
    "file": "000"
  }
}

In [4]:
# Clone config files
def copy_config(src_folder, dest_folder):
    """
    Copies files from src_folder to dest_folder, excluding subfolders.

    Args:
        src_folder: The path to the source folder.
        dest_folder: The path to the destination folder.
    """

    try:
        # Ensure destination folder exists
        os.makedirs(dest_folder, exist_ok=True)

        for filename in os.listdir(src_folder):
            src_path = os.path.join(src_folder, filename)
            dest_path = os.path.join(dest_folder, filename)

            if os.path.isfile(src_path):
                shutil.copy2(src_path, dest_path) #copy metadata as well.
                #Use shutil.copy for not copying metadata.
                print(f"Copied: {filename}")
            #else: #optional
                #print(f"Skipped (not a file): {filename}") #optional. Uncomment if you want to see the skipped folders.

        print("✅ Copying complete.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")


In [5]:
# Copy filtered dataset images/labels
def copy_and_filter_folder(src_folder, dest_folder, pattern):
    """
    Copies a folder and files that match the given pattern.
    Alerts the user when a folder or file already exists but *does not* overwrite.
    Creates only what is needed.

    :param src_folder: Path to the source folder.
    :param dest_folder: Path to the destination folder.
    :param pattern: Filename pattern to keep (e.g., "*.txt").
    """
    try:
        # Ensure destination folder exists
        if not os.path.exists(dest_folder):
            print(f"✓ Creating destination folder '{dest_folder}'.\n")
            os.makedirs(dest_folder)
        else:
            print(f"✓ Destination folder '{dest_folder}' already exists.\n")

        # Walk through the source folder
        for root, _, files in os.walk(src_folder):
            relative_path = os.path.relpath(root, src_folder)
            new_root = os.path.join(dest_folder, relative_path)

            if not os.path.exists(new_root):
                print(f"Creating subdirectory '{new_root}'")
                os.makedirs(new_root)
            else:
                print(f"❕Subdirectory '{new_root}' already exists.")
                print("Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.")

            for file in files:
                if fnmatch.fnmatch(file, pattern + "*"):
                    src_file = os.path.join(root, file)
                    dest_file = os.path.join(new_root, file)

                    if not os.path.exists(dest_file):
                        shutil.copy2(src_file, dest_file)  # copy metadata as well
                    else:
                        print(f"❗️File '{dest_file}' already exists. Skipping.")

            print(f" ✓ Copying files complete.\n")
        print("✅ Copying dataset complete.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")

In [6]:
def save_on_cloud(source: str, destination: str):
    """
    Saves a folder to a cloud storage location (e.g., Google Drive in Colab).

    Args:
        source (str): The path to the source folder.
        destination (str): The path to the destination folder (in cloud storage).
    """
    # 0. Input Validation (Assertions)
    assert isinstance(source, str), "Source must be a string."
    assert isinstance(destination, str), "Destination must be a string."

    try:
        # 1. Verify Source Folder
        if not os.path.exists(destination):
            os.makedirs(destination)

        # 2. Copy the Folder
        shutil.copytree(source, destination, dirs_exist_ok=True)
        print("✅ Folder copied successfully:\n  ",source,"\n  -->",destination)

    except Exception as e:
        print(f"❌ An error occurred: {e}")

# Datasets builder

## Importing from Drive

In [7]:
!rm -rf /content/sample_data

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# Check if the cloud path is ok and the dataset can be found
!ls /content/drive/MyDrive/YOLO

3.5m.v3i.yolov8  3m60.v1i.yolov8  5m60.v1i.yolov8  5m.v1i.yolov8


In [10]:
drive_path = '/content/drive/MyDrive/YOLO'
drive_datasets_paths = os.listdir(drive_path)
drive_datasets = len(drive_datasets_paths)
if (drive_datasets) > 1:
    print("There are %d dataset options:" % drive_datasets)
else:
    print("Theres is only 1 dataset:")
drive_datasets_paths

There are 4 dataset options:


['3.5m.v3i.yolov8', '3m60.v1i.yolov8', '5m.v1i.yolov8', '5m60.v1i.yolov8']

In [12]:
choose_dataset = 4
index = choose_dataset - 1
model_name = os.listdir(drive_path)[index]
print("Choosed model:", model_name)

Choosed model: 5m60.v1i.yolov8


***Readme:***
*   **Option 1:** is desirable if you need to test many subset combinations in the same session (avoid downloading data twice from the cloud).
*   **Option 3:** is best if you're just going to test one subset combination  (avoid downloading unnecessary data from the cloud).



In [13]:
# Option 1 (download full dataset)
source = f"/content/drive/MyDrive/YOLO/{model_name}"
dest = f"/content/YOLO/{model_name}"
#!mkdir /content/YOLO/
!cp -r $source $dest
src_folder = dest

cp: cannot create directory '/content/YOLO/5m60.v1i.yolov8': No such file or directory


In [ ]:
#!mkdir /content/YOLO

In [ ]:
source = "/content/YOLO/3m60.v1i.yolov8.S/valid"
dest = "/content/YOLO/3m60.v1i.yolov8.M/valid"
!mv $source $dest

In [15]:
import shutil
import os

source_dir = source
destination_dir = dest

try:
    shutil.copytree(source_dir, destination_dir)
    print(f"Directory '{source_dir}' copied successfully to '{destination_dir}'")
except FileExistsError:
    print(f"Destination directory '{destination_dir}' already exists.")
    # You can handle this in different ways:
    # - Remove the existing directory and try again:
    # shutil.rmtree(destination_dir)  # Be very careful with this!
    # shutil.copytree(source_dir, destination_dir)
    # - Merge the directories (more complex, requires custom logic)
    # - Prompt the user
except Exception as e:
    print(f"Error copying directory: {e}")


# Example with error handling and overwrite:
def copy_directory(source, destination, overwrite=True):
    try:
        if overwrite and os.path.exists(destination):
            shutil.rmtree(destination)  # Remove existing directory
        shutil.copytree(source, destination)
        print(f"Directory '{source}' copied to '{destination}'")
        return True  # Indicate success
    except FileExistsError:
        print(f"Destination '{destination}' exists. Use overwrite=True to replace.")
        return False  # Indicate failure

Directory '/content/drive/MyDrive/YOLO/5m60.v1i.yolov8' copied successfully to '/content/YOLO/5m60.v1i.yolov8'


In [16]:
copy_directory(source, dest, overwrite=True)

Directory '/content/drive/MyDrive/YOLO/5m60.v1i.yolov8' copied to '/content/YOLO/5m60.v1i.yolov8'


True

## Download model

In [17]:
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolov8n.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 123MB/s]


# Finetuning

### Info

In [18]:
!nvidia-smi

Sun Feb 16 01:00:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [19]:
!yolo version

8.3.75


-----
## Experiment 10 *(full dataset)*
### *5m | 60º | large*

### Train

In [20]:

# Train the model
model.train(
    data=f"/content/YOLO/{model_name}/data.yaml",
    epochs=100,
    imgsz=240,
    freeze=10,
    batch=64
)


Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/YOLO/5m60.v1i.yolov8/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=240, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

100%|██████████| 755k/755k [00:00<00:00, 16.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 101MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[240] must be multiple of max stride 32, updating to [256]


...

                   all        110       1910      0.381      0.349       0.31      0.105



100 epochs completed in 0.060 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.40it/s]


                   all        110       1910      0.376      0.349      0.309      0.105
Speed: 0.0ms preprocess, 0.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x782193c81450>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Validation

In [21]:
# Load the trained YOLO model
model1 = YOLO("/content/runs/detect/train/weights/best.pt")

# Validate the model
model1.val(data=f"/content/YOLO/{model_name}/data.yaml")

Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/YOLO/5m60.v1i.yolov8/valid/labels.cache... 110 images, 3 backgrounds, 0 corrupt: 100%|██████████| 110/110 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:03<00:00,  2.11it/s]


                   all        110       1910      0.384      0.347       0.31      0.104
Speed: 0.4ms preprocess, 2.5ms inference, 0.0ms loss, 7.8ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x782195b8fe90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Inference

#### Test 1 (small weed)

In [22]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [23]:
# Ejecutar predicción sobre la imagen
results = model1(image, save=True, project="model10(5_60_L)", name="experiment1")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg: 256x256 3 plant-weeds, 12.3ms
Speed: 1.6ms preprocess, 12.3ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)
Results saved to model10(5_60_L)/experiment1
Clase: 0, Confianza: 0.74, BBox: (4.17, 161.65, 135.37, 209.77)
Clase: 0, Confianza: 0.48, BBox: (182.02, 185.11, 239.98, 238.40)
Clase: 0, Confianza: 0.43, BBox: (125.25, 198.79, 240.00, 231.30)


In [24]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.875 0.85 0.25416666666666665 0.125
0 0.9041666666666667 0.8875 0.19583333333333333 0.13333333333333333
0 0.9625 0.9458333333333333 0.07916666666666666 0.08333333333333333
0 0.7083333333333334 0.8791666666666667 0.30833333333333335 0.06666666666666667
0 0.4375 0.8458333333333333 0.20833333333333334 0.041666666666666664
0 0.4041666666666667 0.825 0.225 0.041666666666666664
0 0.3333333333333333 0.7583333333333333 0.21666666666666667 0.09166666666666666
0 0.2375 0.7541666666666667 0.225 0.041666666666666664
0 0.09583333333333334 0.7291666666666666 0.19166666666666668 0.09166666666666666



#### Test 2 (medium weed)

In [25]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [26]:
# Ejecutar predicción sobre la imagen
results = model1(image, save=True, project="model10(5_60_L)", name="experiment2")

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg: 256x256 4 plant-weeds, 7.7ms
Speed: 0.9ms preprocess, 7.7ms inference, 1.4ms postprocess per image at shape (1, 3, 256, 256)
Results saved to model10(5_60_L)/experiment2
Clase: 0, Confianza: 0.34, BBox: (22.60, 0.04, 81.20, 53.77)
Clase: 0, Confianza: 0.32, BBox: (0.00, 3.27, 33.06, 75.81)
Clase: 0, Confianza: 0.30, BBox: (0.15, 163.07, 52.65, 207.14)
Clase: 0, Confianza: 0.27, BBox: (28.14, 0.11, 69.57, 45.58)


In [27]:
# Comparing with labeled images
label_file = f'{file_name[:-4]}.txt'
folder_path = image[:image.find(keyword)] + '/labels/'
with open(folder_path+label_file, "r") as file:
    content = file.read()

print(content)

0 0.05416666666666667 0.7958333333333333 0.10416666666666667 0.13333333333333333
0 0.0875 0.6625 0.175 0.275
0 0.25 0.575 0.5 0.125
0 0.09166666666666666 0.5583333333333333 0.10833333333333334 0.35833333333333334
0 0.05416666666666667 0.48333333333333334 0.10416666666666667 0.23333333333333334
0 0.9875 0.25416666666666665 0.025 0.175
0 0.029166666666666667 0.1375 0.058333333333333334 0.275
0 0.1375 0.10416666666666667 0.21666666666666667 0.20833333333333334
0 0.15416666666666667 0.175 0.20833333333333334 0.275



### Save results

In [28]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save/')

✅ Folder copied successfully:
   /content/runs/ 
  --> /content/drive/MyDrive/save/


In [29]:
# Store experiments
save_on_cloud(source='/content/model10(5_60_L)/', destination='/content/drive/MyDrive/save.SM/')

✅ Folder copied successfully:
   /content/model10(5_60_L)/ 
  --> /content/drive/MyDrive/save.SM/
